# Long-Term Episodic Memory with LangGraph

This notebook demonstrates how to implement long-term episodic memory using embeddings and vector search.

## Key Concepts:
- **Episodic memory**: Stores specific events, experiences, and interactions with temporal context
- **Episodes**: Complete narratives of what happened, when, and the outcome
- **Time-aware retrieval**: Retrieve past events relevant to current context
- **Sequential memory**: Preserves the order and relationship between events
- **Contextual recall**: Access rich details about specific moments in history

In [29]:
# Install required dependencies
import subprocess
import sys

packages = [
    "langgraph",
    "langchain",
    "langchain-openai",
    "langchain-chroma",
    "python-dotenv"
]

for package in packages:
    try:
        __import__(package.replace("-", "_"))
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

print("✓ All dependencies installed")

✓ All dependencies installed


In [30]:
import os
from datetime import datetime
from dotenv import load_dotenv
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from typing_extensions import TypedDict

# Verify API key
api_key = " "
if not api_key:
    print("⚠️  Warning: OPENAI_API_KEY not found. Please set it in your .env file")
else:
    print("✓ OpenAI API key loaded")

✓ OpenAI API key loaded


## Step 1: Initialize Embeddings and Vector Store

Setup the episodic memory system with embeddings and a vector database for storing episodes.

In [31]:
try:
    # Initialize embeddings
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=api_key)
    
    # Initialize vector store for episodes
    episode_store = Chroma(
        collection_name="episodic_memory",
        embedding_function=embeddings,
    )
    
    print("✓ Embeddings and vector store initialized")
    
except Exception as e:
    print(f"✗ Error initializing embeddings: {e}")
    print("Make sure OPENAI_API_KEY is set")

✓ Embeddings and vector store initialized


## Step 2: Define the Agent State

The state includes:
- `messages`: Current conversation
- `episodes`: Past events and experiences with temporal context

In [32]:
def add_messages(left: list[BaseMessage], right: list[BaseMessage]) -> list[BaseMessage]:
    """Append messages to existing messages."""
    return left + right

def add_episodes(left: list[str], right: list[str]) -> list[str]:
    """Append episodes to existing episodes."""
    return left + right

class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    episodes: Annotated[list[str], add_episodes]

print("✓ Agent state defined with short-term and episodic memory")

✓ Agent state defined with short-term and episodic memory


## Step 3: Create Agent Nodes

1. **extract_episode**: Extract and record specific events with temporal context
2. **retrieve_episodes**: Find relevant past experiences
3. **agent**: Respond with episode context

In [33]:
try:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, openai_api_key=api_key)
    
    def is_valid_episode(text: str) -> bool:
        """
        Check if extracted text is a real episode, not meta-commentary.
        """
        # Filter out meta-responses
        meta_phrases = [
            "no further episodes",
            "no episodes to extract",
            "it appears that",
            "does not contain",
            "if there were",
            "if you have",
            "please provide",
            "cannot create",
        ]
        
        lower_text = text.lower()
        if any(phrase in lower_text for phrase in meta_phrases):
            return False
        
        # Must have some actual event description
        event_indicators = ["happened", "attended", "met", "experience", "meeting", "conference", "event"]
        return any(indicator in lower_text for indicator in event_indicators)
    
    def extract_episode(state: AgentState) -> dict:
        """
        Extract important episodes (events) from the latest user message.
        Store them with temporal context in the vector store.
        """
        messages = state["messages"]
        
        # Get the last user message
        last_user_msg = None
        for msg in reversed(messages):
            if isinstance(msg, HumanMessage):
                last_user_msg = msg.content
                break
        
        if not last_user_msg:
            return {}
        
        print("📝 Extracting episodes from message...")
        
        # Use LLM to extract events with temporal context
        extraction_prompt = f"""Extract ONLY one main episode (event/experience) from this message if it contains a specific event.

Message: {last_user_msg}

For the episode, include:
- What happened (the event or experience)
- When it happened (relative time: today, last week, last month, recently, etc.)
- Who was involved (people, organizations, etc.)
- The outcome or result

Format as a single clear narrative.
If there is NO specific event or experience mentioned, respond with: "No episodes to extract."
"""
        
        episodes_response = llm.invoke(extraction_prompt)
        episodes_text = episodes_response.content
        
        # Store episode in vector store
        if episodes_text.strip() and "no episodes" not in episodes_text.lower():
            # Split by double newline but filter carefully
            episodes = [e.strip() for e in episodes_text.split("\n\n") if e.strip() and len(e.strip()) > 30]
            
            # Filter out meta-responses
            episodes = [ep for ep in episodes if is_valid_episode(ep)]
            
            if episodes:
                # Add timestamp to episodes for context
                timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                timestamped_episodes = [
                    f"[Timestamp: {timestamp}]\n{ep}" 
                    for ep in episodes
                ]
                print(f"💾 Storing {len(episodes)} episode(s):\n")
                for i, ep in enumerate(timestamped_episodes, 1):
                    print(f"   Episode {i}:")
                    print(f"   ⏰ {ep}")
                    print()
                episode_store.add_texts(timestamped_episodes)
                return {"episodes": episodes}
        
        print("⊘ No episodes extracted from this message")
        return {}
    
    def retrieve_episodes(state: AgentState) -> dict:
        """
        Retrieve relevant episodes from episodic memory based on current context.
        """
        messages = state["messages"]
        
        # Get the last message
        last_msg = messages[-1].content if messages else ""
        
        if not last_msg or len(episode_store.get()["ids"]) == 0:
            return {}
        
        print("🔍 Retrieving relevant episodes from memory...")
        
        # Search for relevant episodes
        results = episode_store.similarity_search(last_msg, k=3)
        
        if results:
            # Filter out meta-commentary from retrieved results
            valid_results = [doc for doc in results if is_valid_episode(doc.page_content)]
            
            if valid_results:
                print(f"✓ Found {len(valid_results)} relevant episode(s):\n")
                for i, doc in enumerate(valid_results, 1):
                    print(f"   Retrieved Episode {i}:")
                    print(f"   📖 {doc.page_content}")
                    print()
                retrieved_episodes = "\n".join([doc.page_content for doc in valid_results])
                return {"episodes": [f"[Past experiences]\n{retrieved_episodes}"]}
            else:
                print("⊘ No relevant episodes found (only metadata)")
                return {}
        
        print("⊘ No relevant episodes found")
        return {}
    
    def agent(state: AgentState) -> dict:
        """
        Agent that processes conversation with episodic memory context.
        """
        messages = state["messages"]
        episodes = state.get("episodes", [])
        
        # Build context from episodes
        context = ""
        if episodes:
            context = f"\nRelevant past experiences:\n" + "\n".join(episodes)
        
        # Prepare system message with episode context
        system_msg = f"""You are a helpful assistant with access to long-term episodic memory.
You remember specific events and experiences from past interactions.
        
{context}

Use the past experiences to provide contextual and personalized responses."""
        
        # Call LLM with full context
        response = llm.invoke([
            {"role": "system", "content": system_msg},
            *messages
        ])
        
        return {"messages": [response]}
    
    print("✓ Agent nodes created successfully")

except Exception as e:
    print(f"✗ Error creating agent nodes: {e}")
    import traceback
    traceback.print_exc()

✓ Agent nodes created successfully


## Step 4: Build the Graph

The workflow:
1. Extract episodes from user message
2. Retrieve relevant past experiences
3. Agent responds with episodic context

In [34]:
# Create the graph
graph_builder = StateGraph(AgentState)

# Add nodes
graph_builder.add_node("extract_episode", extract_episode)
graph_builder.add_node("retrieve_episodes", retrieve_episodes)
graph_builder.add_node("agent", agent)

# Add edges
graph_builder.add_edge(START, "extract_episode")
graph_builder.add_edge("extract_episode", "retrieve_episodes")
graph_builder.add_edge("retrieve_episodes", "agent")
graph_builder.add_edge("agent", END)

# Compile the graph
graph = graph_builder.compile()

print("✓ Graph compiled successfully")

✓ Graph compiled successfully


## Step 5: Run a Multi-Turn Conversation Demo

Demonstrate how episodic memory stores and recalls specific experiences.

In [35]:
try:
    # Clear the episode store for a fresh demo
    print("🗑️  Clearing previous episodes for a fresh start...")
    if len(episode_store.get()["ids"]) > 0:
        episode_store.delete(episode_store.get()["ids"])
        print(f"✓ Cleared {len(episode_store.get()['ids'])} old episodes\n")
    else:
        print("✓ Store is empty\n")
    
    # Initialize conversation with episodic memory
    initial_state = {
        "messages": [],
        "episodes": []
    }
    
    # Multi-turn conversation with episodic events
    conversation_turns = [
        "Last month I attended a AI conference in Berlin where I met amazing people working on LLMs.",
        "Tell me about my recent experiences.",
        "Yesterday I had a challenging meeting with my team about a failed deployment.",
        "How did my experiences shape my current situation?"
    ]
    
    print("=" * 60)
    print("LONG-TERM EPISODIC MEMORY DEMONSTRATION")
    print("=" * 60)
    print()
    
    current_state = initial_state.copy()
    
    for i, user_input in enumerate(conversation_turns, 1):
        print(f"Turn {i}: User Input")
        print(f">> {user_input}")
        print()
        
        # Add user message to state
        current_state["messages"].append(HumanMessage(content=user_input))
        
        # Run the agent
        result = graph.invoke(current_state)
        current_state = result
        
        # Extract and display agent response
        last_message = current_state["messages"][-1]
        print(f"Turn {i}: Agent Response")
        print(f"<< {last_message.content}")
        print()
        
        # Show episodic memory size
        stored_count = len(episode_store.get()["ids"])
        print(f"📚 Episodes stored in memory: {stored_count}")
        print("-" * 60)
        print()
    
    print("✓ Multi-turn conversation completed")
    print(f"✓ Total episodes stored in episodic memory: {len(episode_store.get()['ids'])}")

except Exception as e:
    print(f"✗ Error running conversation: {e}")
    import traceback
    traceback.print_exc()

🗑️  Clearing previous episodes for a fresh start...
✓ Cleared 0 old episodes

LONG-TERM EPISODIC MEMORY DEMONSTRATION

Turn 1: User Input
>> Last month I attended a AI conference in Berlin where I met amazing people working on LLMs.

📝 Extracting episodes from message...
💾 Storing 1 episode(s):

   Episode 1:
   ⏰ [Timestamp: 2025-12-11 22:05:18]
Last month, I attended an AI conference in Berlin where I met amazing people working on LLMs. The event allowed me to network with professionals in the field, enhancing my understanding of language models and their applications.

🔍 Retrieving relevant episodes from memory...
✓ Found 1 relevant episode(s):

   Retrieved Episode 1:
   📖 [Timestamp: 2025-12-11 22:05:18]
Last month, I attended an AI conference in Berlin where I met amazing people working on LLMs. The event allowed me to network with professionals in the field, enhancing my understanding of language models and their applications.

Turn 1: Agent Response
<< That sounds like a fantas

## Key Takeaways: Long-Term Episodic Memory Pattern

### How it Works:
1. **Extract**: Specific events and experiences are extracted from user messages
2. **Timestamp**: Events are timestamped to preserve temporal context
3. **Store**: Rich episode narratives are stored in the vector database
4. **Retrieve**: Relevant past experiences are retrieved based on current context
5. **Contextualize**: Episodes augment the agent's understanding of the user's journey

### Characteristics:
- ✅ Remembers specific events with full context
- ✅ Preserves temporal information (when things happened)
- ✅ Retrieves semantically similar experiences
- ✅ Better than semantic memory for narrative understanding
- ✅ Helps agent understand cause-and-effect relationships
- ❌ More verbose than semantic memory (longer storage)
- ❌ Requires event extraction logic

### Differences from Semantic Memory:
| Aspect | Semantic | Episodic |
|--------|----------|----------|
| **Content** | Facts and knowledge | Events and experiences |
| **Temporal** | Timeless | Time-aware |
| **Context** | What is known | What happened and when |
| **Use case** | General knowledge | Personal history |

### Common Use Cases:
- Personal diary or journal assistant
- Mentoring systems tracking student progress
- Customer service with interaction history
- Project management tracking milestones
- Therapy or coaching with session notes